In [4]:
import pandas as pd
import numpy as np
import os


def target_ipo(ipos, rounds,companies,reference="a"):
    rounds.funded_at = pd.to_datetime(rounds.funded_at)
    ipos.public_at = pd.to_datetime(ipos.public_at)

    ipos_rounds = ipos[["object_id","public_at"]].merge(rounds, on="object_id", how="inner")

    tmp1 = ipos_rounds[ipos_rounds.funding_round_code == reference].sort_values(by="funded_at")\
    .groupby("object_id",as_index=False).first()
    tmp2 = ipos_rounds[ipos_rounds.funding_round_type == f'series-{reference}']\
    .sort_values(by="funded_at").groupby("object_id",as_index=False).first()
    tmp3 = pd.concat([tmp1,tmp2]).drop_duplicates('object_id')\
    .rename(columns={'funded_at':f"date_series_{reference}"})

    tmp3 = tmp3.drop(columns=["id","funding_round_id","raised_amount","raised_currency_code","is_first_round",\
    "is_last_round","source_url","source_description","created_by","created_at", "updated_at",\
    "post_money_valuation","post_money_currency_code", "pre_money_valuation_usd", "pre_money_currency_code",\
    "post_money_valuation_usd","pre_money_valuation","funding_round_type"])
    tmp3 = tmp3.rename(columns={"object_id":"id","raised_amount_usd":f"raised_amount_usd_{reference}",\
                        "participants":f"participants_{reference}", "public_at":"exit_date"})
    tmp3["funding_round_code"] = reference
    tmp3["exit"]="ipo"

    companies = companies.merge(tmp3, how="inner", on="id") #.sort_values(by="public_at")
    print(companies.head())

    return companies

def target_acq(acq, rounds,companies,reference="a"):
    acq.acquired_at = pd.to_datetime(acq.acquired_at)
    acq = acq.rename(columns={"acquired_object_id":"object_id"})
    rounds.funded_at = pd.to_datetime(rounds.funded_at)

    acq_rounds = acq[["object_id","acquired_at"]].merge(rounds, on="object_id", how="inner")

    tmp1 = acq_rounds[acq_rounds.funding_round_code == reference].sort_values(by="funded_at")\
    .groupby("object_id",as_index=False).first()
    tmp2 = acq_rounds[acq_rounds.funding_round_type == f'series-{reference}']\
    .sort_values(by="funded_at").groupby("object_id",as_index=False).first()
    tmp3 = pd.concat([tmp1,tmp2]).drop_duplicates('object_id')\
    .rename(columns={'funded_at':f"date_series_{reference}"})

    tmp3 = tmp3.drop(columns=["id","funding_round_id","raised_amount","raised_currency_code","is_first_round",\
    "is_last_round","source_url","source_description","created_by","created_at", "updated_at",\
    "post_money_valuation","post_money_currency_code", "pre_money_valuation_usd", "pre_money_currency_code",\
    "post_money_valuation_usd","pre_money_valuation","funding_round_type"])
    tmp3 = tmp3.rename(columns={"object_id":"id","raised_amount_usd":f"raised_amount_usd_{reference}",\
                        "participants":f"participants_{reference}","acquired_at":"exit_date"})
    tmp3["funding_round_code"] = reference
    tmp3["exit"]="acquisition"

    companies = companies.merge(tmp3, how="inner", on="id") #.sort_values(by="acquired_at")
    print(companies.head())

    return companies

def target_no_exit(rounds, companies, reference="a"):
    no_rounds = companies.merge(rounds, on="object_id", how="inner")
    tmp1 = no_rounds[no_rounds.funding_round_code == reference].sort_values(by="funded_at")\
    .groupby("object_id",as_index=False).first()
    tmp2 = no_rounds[no_rounds.funding_round_type == f'series-{reference}']\
    .sort_values(by="funded_at").groupby("object_id",as_index=False).first()
    tmp3 = pd.concat([tmp1,tmp2]).drop_duplicates('object_id')\
    .rename(columns={'funded_at':f"date_series_{reference}"})

    tmp3 = tmp3.drop(columns=["id","funding_round_id","raised_amount","raised_currency_code","is_first_round",\
    "is_last_round","source_url","source_description","created_by","created_at", "updated_at",\
    "post_money_valuation","post_money_currency_code", "pre_money_valuation_usd", "pre_money_currency_code",\
    "post_money_valuation_usd","pre_money_valuation","funding_round_type"])
    tmp3 = tmp3.rename(columns={"object_id":"id","raised_amount_usd":f"raised_amount_usd_{reference}",\
                        "participants":f"participants_{reference}"})
    tmp3["funding_round_code"] = reference
    tmp3["exit"]="no exit"
    tmp3["exit_date"]=np.nan
    print(tmp3.head())

    return tmp3



def get_company_target(ipos, acq, rounds,companies,reference="a"):
    companies_ipo = target_ipo(ipos, rounds,companies,reference).set_index("id")
    companies_acq = target_acq(acq, rounds, companies, reference).set_index("id")
    companies_ipo_acq  = pd.concat([companies_ipo,companies_acq], axis=0)
    rows_exit = companies_ipo_acq.index
    companies_no_exit = companies.set_index('id').drop(rows_exit).reset_index().rename(columns={"id":"object_id"})
    companies_no_exit = target_no_exit(rounds, companies_no_exit,reference).set_index("id")

    companies = pd.concat([companies_ipo_acq,companies_no_exit], axis=0)

    print(companies.sample(20))

    return companies






















ipos = pd.read_csv(os.path.join('..',"raw_data","ipos.csv"))
acq = pd.read_csv(os.path.join('..',"raw_data","acquisitions.csv"))
rounds = pd.read_csv(os.path.join('..',"raw_data","funding-rounds.csv"))
companies = pd.read_csv(os.path.join('..',"raw_data","companies.csv"))

companies=get_company_target(ipos, acq, rounds,companies,reference="a")
#print(companies.head())
print(companies.shape)

         id             name  normalized_name                 permalink  \
0   c:10241  Global Crossing  global crossing  /company/global-crossing   
1  c:104377     Tremor Video     tremor video     /company/tremor-video   
2    c:1063             YuMe             yume             /company/yume   
3   c:10704            Chegg            chegg            /company/chegg   
4    c:1105              Wix              wix              /company/wix   

      category_code    status  founded_at closed_at  \
0  public_relations  acquired  1997-01-01       NaN   
1       advertising       ipo  2005-01-01       NaN   
2       advertising       ipo  2004-01-01       NaN   
3         education       ipo  2005-01-01       NaN   
4               web       ipo  2006-01-01       NaN   

                    homepage_url twitter_username  ... funding_rounds  \
0  http://www.globalcrossing.com   GlobalCrossing  ...              1   
1         http://tremorvideo.com      tremorvideo  ...              8   

category_code => boxplot 
status ... IPO ou autres à plot 
funding_rounds plot
funding_total_usd nuage 


In [33]:
rounds.value_counts()

id     funding_round_id  object_id  funded_at   funding_round_type  funding_round_code  raised_amount_usd  raised_amount  raised_currency_code  pre_money_valuation_usd  pre_money_valuation  pre_money_currency_code  post_money_valuation_usd  post_money_valuation  post_money_currency_code  participants  is_first_round  is_last_round  source_url                                                                                                                                               source_description                                                                                                          created_by        created_at           updated_at         
57952  57952             c:286215   2010-04-07  venture             partial             271250.0           271250.0       USD                   0.0                      0.0                  USD                      0.0                       0.0                   USD                       0             1               1          

In [30]:
companies['category_code'].count


<bound method Series.count of id
c:10241     public_relations
c:104377         advertising
c:1063           advertising
c:10704            education
c:1105                   web
                  ...       
c:9489      public_relations
c:94897          photo_video
c:95330            ecommerce
c:9812             ecommerce
c:998                    web
Name: category_code, Length: 8854, dtype: object>

In [35]:
!pip install matplotlib
import matplotlib.pyplot as plt

     |████████████████████████████████| 8.5 MB 2.5 MB/s eta 0:00:01
  Using cached certifi-2020.6.20-py2.py3-none-any.whl (156 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
     |████████████████████████████████| 2.2 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 11.3 MB/s eta 0:00:01


In [38]:
plt.plot

AttributeError: module 'matplotlib.pyplot' has no attribute 'companies'

In [19]:
ipos


,ipo_id,object_id,valuation_amount,valuation_currency_code,stock_symbol,source_url,source_description,public_at
0,1,c:1654,NaN,USD,NASDAQ:AAPL,NaN,NaN,1980-12-19
1,2,c:1242,NaN,USD,NASDAQ:MSFT,NaN,NaN,1986-03-13
2,3,c:342,NaN,USD,NYSE:DIS,NaN,NaN,1969-06-09
3,4,c:59,NaN,USD,NASDAQ:GOOG,NaN,NaN,2004-08-25
4,5,c:317,1.000000e+11,USD,NASDAQ:AMZN,NaN,NaN,1997-05-01
...,...,...,...,...,...,...,...,...
1253,1372,c:267859,NaN,USD,NYSE:EBS,NaN,NaN,2006-11-15
1254,1373,c:39330,NaN,USD,NYSE:ARMK,http://www.reuters.com/article/2013/12/11/aram...,Food services firm Aramark raises $725 mln in ...,2013-12-11
1255,1374,c:220208,NaN,USD,NYSE:HLT,http://news.hiltonworldwide.com/index.cfm/news...,Hilton Worldwide Prices Initial Public Offering,2008-12-08
1256,1375,c:243568,NaN,USD,OTC:BCLI,NaN,NaN,2003-10-07


In [20]:
acq

,acquired_object_id,acquisition_id,acquiring_object_id,term_code,price_amount,price_currency_code,acquired_at,source_url,source_description
0,c:10,1,c:11,NaN,20000000.0,USD,2007-05-30,http://venturebeat.com/2007/05/30/fox-interact...,Fox Interactive confirms purchase of Photobuck...
1,c:100,20,c:377,cash,NaN,USD,2005-05-29,http://www.there.com/pr_acquisition.html,Makena Technologies Acquires There from Forter...
2,c:1001,1901,c:5,cash_and_stock,47500000.0,USD,2009-08-10,http://www.techcrunch.com/2009/08/10/facebook-...,Facebook Acquires FriendFeed
3,c:10014,3878,c:23054,NaN,NaN,USD,2010-09-30,http://techcrunch.com/2010/09/30/mobclix-acqui...,Mobclix Acquired By UK Mobile Marketing Compan...
4,c:100265,6106,c:38215,NaN,NaN,USD,2011-09-06,http://www.businesswire.com/news/home/20110906...,Harvest Power Acquires Coastal Supply
...,...,...,...,...,...,...,...,...,...
9406,c:9949,9692,c:267149,NaN,400000000.0,USD,2013-09-25,http://www.techinasia.com/report-suning-buy-pp...,Report: Suning to buy PPTV for around $400 mil...
9407,c:99685,7240,c:161312,stock,4800000.0,USD,2011-08-01,NaN,NaN
9408,c:997,85,c:29,cash,350000000.0,USD,2007-09-01,http://www.techcrunch.com/2007/09/17/breaking-...,Breaking: Yahoo Acquires Zimbra For $350 milli...
9409,c:99737,6463,c:67724,NaN,NaN,USD,2011-11-30,http://finance.yahoo.com/news/west-corporation...,West Corporation to Acquire HyperCube


In [21]:
rounds

,id,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,...,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round,source_url,source_description,created_by,created_at,updated_at
0,1,1,c:4,2006-12-01,series-b,b,8500000.0,8500000.0,USD,0.0,...,0.0,NaN,2,0,0,http://www.marketingvox.com/archives/2006/12/2...,NaN,initial-importer,2007-07-04 04:52:57,2008-02-27 23:14:29
1,2,2,c:5,2004-09-01,angel,angel,500000.0,500000.0,USD,0.0,...,0.0,USD,2,0,1,NaN,NaN,initial-importer,2007-05-27 06:08:18,2013-06-28 20:07:23
2,3,3,c:5,2005-05-01,series-a,a,12700000.0,12700000.0,USD,115000000.0,...,0.0,USD,3,0,0,http://www.techcrunch.com/2007/11/02/jim-breye...,Jim Breyer: Extra $500 Million Round For Faceb...,initial-importer,2007-05-27 06:09:10,2013-06-28 20:07:23
3,4,4,c:5,2006-04-01,series-b,b,27500000.0,27500000.0,USD,525000000.0,...,0.0,USD,4,0,0,http://www.facebook.com/press/info.php?factsheet,Facebook Funding,initial-importer,2007-05-27 06:09:36,2013-06-28 20:07:24
4,5,5,c:7299,2006-05-01,series-b,b,10500000.0,10500000.0,USD,0.0,...,0.0,NaN,2,0,0,http://www.techcrunch.com/2006/05/14/photobuck...,PhotoBucket Closes $10.5M From Trinity Ventures,initial-importer,2007-05-29 11:05:59,2008-04-16 17:09:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52923,57948,57948,c:211890,2013-12-12,series-a,a,3000000.0,3000000.0,USD,0.0,...,0.0,USD,1,1,1,http://techcrunch.com/2013/12/12/bitstrips-ser...,"Bitstrips Confirms $3M Series A From Horizons,...",amit2013,2013-12-12 14:12:05,2013-12-12 14:12:05
52924,57949,57949,c:267427,2010-02-06,venture,partial,570000.0,570000.0,USD,0.0,...,0.0,USD,0,0,1,http://www.sec.gov/Archives/edgar/data/1434188...,SEC,santosh2013,2013-12-12 14:13:11,2013-12-12 14:13:11
52925,57950,57950,c:261728,2010-02-06,venture,unattributed,2184100.0,2184100.0,USD,0.0,...,0.0,USD,0,0,1,http://www.sec.gov/Archives/edgar/data/1335158...,SEC,santosh2013,2013-12-12 14:19:35,2013-12-12 14:19:35
52926,57951,57951,c:285864,2013-12-12,series-a,a,790783.0,790783.0,USD,0.0,...,0.0,USD,0,1,1,http://www.startupticker.ch/en/news/december-2...,UrbanFarmers closes interim-investment round,amit2013,2013-12-12 14:23:49,2013-12-12 14:23:49


In [22]:
companies

,name,normalized_name,permalink,category_code,status,founded_at,closed_at,homepage_url,twitter_username,overview,...,funding_rounds,funding_total_usd,description,country_code_with_US,exit_date,date_series_a,funding_round_code,raised_amount_usd_a,participants_a,exit
id,,,,,,,,,,,,,,,,,,,,,
c:10241,Global Crossing,global crossing,/company/global-crossing,public_relations,acquired,1997-01-01,NaN,http://www.globalcrossing.com,GlobalCrossing,Global Crossing Limited (GCL) is a communicati...,...,1,41000000.0,public_relations IP Telecommunications Globa...,USA NJ,NaT,1997-03-01,a,41000000.0,1,ipo
c:104377,Tremor Video,tremor video,/company/tremor-video,advertising,ipo,2005-01-01,NaN,http://tremorvideo.com,tremorvideo,Tremor Video is a digital video technology com...,...,8,116400000.0,advertising Tremor Video is a digital vide...,USA NY,2013-06-27 00:00:00,2006-09-01,a,8400000.0,2,ipo
c:1063,YuMe,yume,/company/yume,advertising,ipo,2004-01-01,NaN,http://www.yume.com,YuMeVideo,"YuMe, Inc. is a leading provider of digital vi...",...,8,72900000.0,advertising YuMe provides digital video advert...,USA CA,2013-07-02 00:00:00,2007-03-05,a,7000000.0,3,ipo
c:10704,Chegg,chegg,/company/chegg,education,ipo,2005-01-01,NaN,http://www.chegg.com,chegg,"Chegg, an online textbook rental company, was ...",...,10,252303692.0,"education Online textbook rental Chegg, an o...",USA CA,2013-11-14 00:00:00,2007-01-01,a,2200000.0,2,ipo
c:1105,Wix,wix,/company/wix,web,ipo,2006-01-01,NaN,http://www.wix.com,Wix,"\nWix.com Ltd., (www.wix.com) is a leading clo...",...,4,58500000.0,web Wix is a DIY web publishing platform enabl...,ISR,2013-11-06 00:00:00,2007-12-01,a,5000000.0,2,ipo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c:9489,RoyaltyShare,royaltyshare,/company/royaltyshare,public_relations,operating,2005-01-01,NaN,http://royaltyshare.com,royaltyshare,RoyaltyShare provides Web-based royalty proces...,...,4,11500000.0,public_relations RoyaltyShare provides Web...,USA CA,NaN,2006-07-31,a,2500000.0,1,no exit
c:94897,Trover,trover,/company/trover,photo_video,operating,2010-01-01,NaN,http://trover.com,trover,Trover is a little team of 6 impassioned peopl...,...,1,2500000.0,photo_video Trover is an information sharing p...,USA WA,NaN,2013-07-25,a,2500000.0,4,no exit
c:95330,Colingo,colingo,/company/colingo,ecommerce,operating,2011-01-01,NaN,http://www.colingo.com,colingo,Colingo was founded in 2011 to break down barr...,...,1,2400000.0,ecommerce Colingo was founded in 2011 to b...,USA CA,NaN,2013-11-20,a,2400000.0,6,no exit


In [7]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8854 entries, c:10241 to c:998
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   name                  8854 non-null   object        
 1   normalized_name       8854 non-null   object        
 2   permalink             8854 non-null   object        
 3   category_code         8635 non-null   object        
 4   status                8854 non-null   object        
 5   founded_at            7334 non-null   object        
 6   closed_at             530 non-null    object        
 7   homepage_url          8577 non-null   object        
 8   twitter_username      5969 non-null   object        
 9   overview              8814 non-null   object        
 10  tag_list              4544 non-null   object        
 11  country_code          8538 non-null   object        
 12  state_code            6023 non-null   object        
 13  city            

In [18]:
companies['name']

id
c:10241     Global Crossing
c:104377       Tremor Video
c:1063                 YuMe
c:10704               Chegg
c:1105                  Wix
                 ...       
c:9489         RoyaltyShare
c:94897              Trover
c:95330             Colingo
c:9812               nooked
c:998                 Wamba
Name: name, Length: 8854, dtype: object